# Tutorial 4 - Hierarchical Agents

## Functionalities:
- An agent (referred as meta agent) can also assign other agents (referred as inner agents) as the function call, thereby creating a hierarchical structure
- Whenever the inner agent is called, we will run the full inner agent's process until termination
- Maximum number of steps for each agent (meta agent or inner agent) can be defined in `max_subtasks` (default: 5) variable when initialising agent

## Additional Details:
- The inner agent will take in meta agent's instruction as input
- In addition, the inner agent will also have access to meta agent's task and subtasks completed for global context
- Inner agent's output will directly go into meta agent's subtasks completed dictionary, enabling it to directly update global context

## Design Philosophy:
- Bottom layers can get context of the top layers for more context-specific generation
- Top layers choose bottom layers in a hierarchical fashion so as to limit the number of choices at each level for better performance
- `subtasks_completed` serves as the global context for the meta agents and the inner agents, and can be summarised as needed, e.g. when performing a new task to filter out irrelevant information 

# Setup Guide

## Step 1: Install TaskGen

In [1]:
# !pip install taskgen-ai

## Step 2: Import required functions and setup relevant API keys for your LLM

In [2]:
# Set up API key and do the necessary imports
from taskgen import *
import os

# this is only if you use OpenAI as your LLM
os.environ['OPENAI_API_KEY'] = '<YOUR API KEY HERE>'

## Step 3: Define your own LLM
- Take in a `system_prompt`, `user_prompt`, and outputs llm response string
- Note: If you do not define your own LLM, OpenAI 'gpt-3.5-turbo' will be used as default

In [3]:
def llm(system_prompt: str, user_prompt: str) -> str:
    ''' Here, we use OpenAI for illustration, you can change it to your own LLM '''
    # ensure your LLM imports are all within this function
    from openai import OpenAI
    
    # define your own LLM here
    client = OpenAI()
    response = client.chat.completions.create(
        model='gpt-3.5-turbo',
        temperature = 0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content

In [4]:
# Verify that llm function is working
llm(system_prompt = 'You are a classifier to classify the sentiment of a sentence', 
    user_prompt = 'It is a hot and sunny day')

'Positive'

# Define Agents and Meta Agents
- You can also create a Meta Agent that uses other Agents (referred to as Inner Agents) as functions
- Create your Meta agent using `Agent()` (Note: No different from usual process of creating Agents - your Meta Agent is also an Agent)
- Set up an Inner Agent list and assign it to your Meta agent using `assign_agents(agent_list)`

## Example Meta Agent Setup
```python
# Define your meta-agent
my_agent = Agent('Menu Creator', 
                 'Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.',
                 llm = llm)

# Define your agent list. Note you can just assign functions to the agent in place using .assign_functions(function_list)
agent_list = [
    Agent('Chef', 'Takes in dish names and generates ingredients for each of them. Does not generate prices.', llm = llm),
    Agent('Boss', 'Makes decision on which items to include in menu.', llm = llm),
    Agent('Creative Writer', 'Takes in a cuisine type and generates interesting dish names and descriptions.', max_subtasks = 2, llm = llm),
    Agent('Economist', 'Takes in dish names and comes up with pricing for each of them', llm = llm).assign_functions([fp])
    ]

my_agent.assign_agents(agent_list)
```

## Run the Meta Agent
- Let us run the agent and see the interactions between the Meta Agent and Inner Agents to solve the task!
```python
output = my_agent.run('Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Ensure all parts of menu are generated.')
```

In [5]:
def random_number_from_string(input_string) -> int:
    ''' Returns a random number between 1 and 10 when given an input string '''
    import hashlib
    import random
    # Hash the input string
    hashed_string = hashlib.sha256(input_string.encode()).hexdigest()
    
    # Convert the hashed string to an integer
    hashed_integer = int(hashed_string, 16)
    
    # Seed the random number generator with the hashed integer
    random.seed(hashed_integer)
    
    # Generate a random number between 1 and 10
    return random.randint(1, 10)

In [6]:
# assign a dish pricing function for economist
# Note: Try to make the output informative that is understandable by the Agent - use free text or json to explain the output so that someone can understand without the input
def dish_price(list_of_dish_names: list) -> dict:
    ''' Takes in list_of_dish_names and outputs price of each dish '''
    if not isinstance(list_of_dish_names, list):
        list_of_food_items = [list_of_dish_names]
    output = {}
    for each in list_of_dish_names:
        output[each] = '$'+str(random_number_from_string(each))
    return output

In [7]:
# Define your meta-agent
my_agent = Agent('Menu Creator', 
                 'Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.',
                llm = llm)

In [8]:
# Define your agent list. Note you can just assign functions to the agent in place using .assign_functions(function_list)
agent_list = [
    Agent('Chef', 'Takes in dish names and generates ingredients for each of them. Does not generate prices.', llm = llm),
    Agent('Boss', 'Makes decisions on which items to include in Menu', llm = llm),
    Agent('Creative Writer', 'Takes in a cuisine type and generates interesting dish names and descriptions. Does not generate prices or ingredients.', max_subtasks = 1, llm = llm),
    Agent('Economist', 'Takes in dish names and comes up with pricing for each of them', llm = llm).assign_functions([dish_price])
    ]

In [9]:
my_agent.assign_agents(agent_list)

# Approach 1: Run a generic task with the Meta-Agent
- Let us run a general task to create 5 menu items for an Italian restaurant
- You can also specify how Agents interact with one another in the task, otherwise the Meta-Agent will infer what should be done

In [10]:
output = my_agent.run('Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Output in Menu format.')

Observation: No subtasks have been completed yet for the assigned task.
Thoughts: To generate 5 menu items with name, description, ingredients, and price based on Italian food choices, we will need to utilize the Equipped Functions available to create dish names, descriptions, ingredients, and pricing for each menu item.
Subtask identified: Generate interesting dish names and descriptions for Italian food choices.
Calling function Creative Writer with parameters {'instruction': 'Generate interesting dish names and descriptions for Italian food choices.'}

### Start of Inner Agent: Creative Writer ###
Observation: No subtasks have been completed yet for the Assigned Task.
Thoughts: To complete the Assigned Task, I need to brainstorm interesting dish names and descriptions for Italian food choices.
Subtask identified: Brainstorm interesting dish names and descriptions for Italian food choices such as 'Pasta alla Fantasia - A whimsical blend of flavors in every bite' or 'Tiramisu Twist - 

In [11]:
output = my_agent.reply_user()

For the Italian food choices, here are 5 menu items created: 1. Risotto Romantico - Creamy Arborio rice infused with saffron and topped with grilled shrimp, a dish that will sweep you off your feet. Ingredients: Arborio rice, saffron, shrimp. Price: $15. 2. Pizza Primavera - A colorful medley of fresh vegetables on a thin crust, a taste of spring in every slice. Ingredients: Fresh vegetables, thin crust. Price: $12. 3. Cannoli Crunch - Crispy pastry tubes filled with sweet ricotta and chocolate chips, a delightful treat with a satisfying crunch. Ingredients: Pastry tubes, ricotta, chocolate chips. Price: $8. 4. Linguine al Pesto - Al dente linguine pasta tossed in a fragrant basil pesto sauce, a classic Italian favorite. Ingredients: Linguine pasta, basil, pine nuts, Parmesan cheese. Price: $14. 5. Tiramisu Temptation - Layers of espresso-soaked ladyfingers and mascarpone cream dusted with cocoa, a heavenly dessert. Ingredients: Ladyfingers, mascarpone cheese, espresso, cocoa. Price: $

In [12]:
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Shared Variables: ['agent']
Task: Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Output in Menu format.
Subtasks Completed:
Subtask: Creative Writer(instruction="Generate interesting dish names and descriptions for Italian food choices.")
{'Agent Output': 'For Italian food choices, here are some interesting dish names and descriptions: 1. Risotto Romantico - Creamy Arborio rice infused with saffron and topped with grilled shrimp, a dish that will sweep you off your feet. 2. Pizza Primavera - A colorful medley of fresh vegetables on a thin crust, a taste of spring in every slice. 3. Cannoli Crunch - Crispy pastry tubes filled with sweet ricotta and chocolate chips, a delightful treat with a satisfying crunch.'

# Approach 2: Step through the process yourself as the Meta Agent
- You can use agents manually using `use_agent(agent_name: str, agent_task: str)` The outputs of the agents will be stored into `subtasks_completed` of the meta agent automatically
- You can also interact with the meta agent using `reply_user(task: str, stateful: bool = False)`. This generates a reply for the agent to the `task` using `subtasks_completed` as context. If `stateful` is true, this will also store the output into `subtasks_completed`
- If `subtasks_completed` is too lengthy, you can run `summarise_subtasks_completed(task: str)` This summarises `subtasks_completed` according to what is relevant to the `task`
- If you want to redo a subtask, you can remove the most recent subtask from `subtasks_completed` using `remove_last_subtask()`

In [13]:
# reset the state of the agent
my_agent.reset()
# currently should have no task and nothing in subtasks_completed
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Shared Variables: ['agent']
Task: No task assigned
Subtasks Completed: None
Is Task Completed: False


In [14]:
# Assign a task for the meta agent - this helps to contextualise what you give each of the inner agents later on
my_agent.assign_task('Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Output in Menu format.')

In [15]:
# This will run the Agent Creative Writer (with shared subtasks_completed as meta agent)
# Output of task will go directly into subtasks_completed - we view the inner agent as simply an extension of the meta agent
my_agent.use_agent('Creative Writer', 'Generate only 5 Italian dish names and descriptions')

Calling function Creative Writer with parameters {'instruction': 'Generate only 5 Italian dish names and descriptions'}

### Start of Inner Agent: Creative Writer ###
Observation: No subtasks have been completed yet for the assigned task.
Thoughts: To complete the assigned task, I need to come up with 5 Italian dish names and descriptions.
Subtask identified: Generate the first Italian dish name and description.
Getting LLM to perform the following task: Generate the first Italian dish name and description.
> The Italian dish name is "Rustic Tuscan Chicken" with a description: A hearty and flavorful dish inspired by the Tuscan countryside. Tender chicken cooked in a rich tomato and herb sauce, served with a side of creamy polenta and roasted vegetables. A comforting and satisfying meal that brings the taste of Italy to your table.

###
Reply from Creative Writer to Menu Creator:
1. Rustic Tuscan Chicken: A hearty and flavorful dish inspired by the Tuscan countryside. Tender chicken coo

In [16]:
# subtasks_completed should be updated with the Creative Agent's outputs
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Shared Variables: ['agent']
Task: Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Output in Menu format.
Subtasks Completed:
Subtask: Creative Writer(instruction="Generate only 5 Italian dish names and descriptions")
{'Agent Output': '1. Rustic Tuscan Chicken: A hearty and flavorful dish inspired by the Tuscan countryside. Tender chicken cooked in a rich tomato and herb sauce, served with a side of creamy polenta and roasted vegetables. A comforting and satisfying meal that brings the taste of Italy to your table.\n2. Classic Margherita Pizza: A traditional Italian pizza featuring a thin crust topped with fresh tomatoes, mozzarella cheese, basil leaves, and a drizzle of olive oil. A timeless favorite that capt

In [17]:
my_agent.use_agent('Chef', 'Generate ingredients for the 5 Italian dishes')

Calling function Chef with parameters {'instruction': 'Generate ingredients for the 5 Italian dishes'}

### Start of Inner Agent: Chef ###
Observation: The dish names and descriptions for the 5 Italian dishes have already been generated by the Creative Writer. No further information is provided on the ingredients for the dishes.
Thoughts: To complete the task of generating ingredients for the 5 Italian dishes, it is essential to research and compile the list of ingredients typically used in each dish based on traditional Italian recipes.
Subtask identified: Research and compile a list of ingredients typically used in Rustic Tuscan Chicken, Classic Margherita Pizza, Creamy Pesto Pasta, Sicilian Caponata, and Tiramisu Delight.
Getting LLM to perform the following task: Research and compile a list of ingredients typically used in Rustic Tuscan Chicken, Classic Margherita Pizza, Creamy Pesto Pasta, Sicilian Caponata, and Tiramisu Delight.
> Rustic Tuscan Chicken: Ingredients typically used

In [18]:
my_agent.use_agent('Economist', 'Generate prices for the 5 Italian dishes')

Calling function Economist with parameters {'instruction': 'Generate prices for the 5 Italian dishes'}

### Start of Inner Agent: Economist ###
Observation: No subtasks have been completed yet for the Assigned Task. The dish names and descriptions have been provided along with the ingredients for the dishes.
Thoughts: To generate prices for the 5 Italian dishes, we need to utilize the dish names and descriptions provided to come up with suitable pricing for each dish. It would be beneficial to consider the complexity, ingredients, and market value of each dish to determine appropriate prices.
Subtask identified: Use the dish names and descriptions to estimate prices for each dish based on factors such as ingredients, complexity, and market value.
Calling function dish_price with parameters {'list_of_dish_names': ['Rustic Tuscan Chicken', 'Classic Margherita Pizza', 'Creamy Pesto Pasta', 'Sicilian Caponata', 'Tiramisu Delight']}
> {'output_1': {'Rustic Tuscan Chicken': '$3', 'Classic Ma

In [19]:
my_agent.use_agent('Boss', 'Choose only 3 Italian dishes to serve that will please Gordon Ramsay')

Calling function Boss with parameters {'instruction': 'Choose only 3 Italian dishes to serve that will please Gordon Ramsay'}

### Start of Inner Agent: Boss ###
Observation: No subtasks have been completed yet for the assigned task. The task involves selecting 3 Italian dishes to serve that will please Gordon Ramsay.
Thoughts: Since no subtasks have been completed, it is essential to first review the Italian dishes available and their descriptions, ingredients, and prices to determine which 3 dishes would be most appealing to Gordon Ramsay.
Subtask identified: Review the descriptions, ingredients, and prices of the 5 Italian dishes provided to select the 3 dishes that are likely to please Gordon Ramsay.
Getting LLM to perform the following task: Review the descriptions, ingredients, and prices of the 5 Italian dishes provided to select the 3 dishes that are likely to please Gordon Ramsay.
> Based on the descriptions, ingredients, and prices provided for the 5 Italian dishes, the 3 dis

In [20]:
# Get a targeted response from your agent, and add this to subtask_completed
output = my_agent.reply_user('Choose the 3 menu items for Gordon Ramsay line by line in this format - Dish Name, Dish Description, Dish Ingredients, Dish Price')

Classic Margherita Pizza, A traditional Italian pizza featuring a thin crust topped with fresh tomatoes, mozzarella cheese, basil leaves, and a drizzle of olive oil, Tomatoes, Mozzarella cheese, Basil leaves, Olive oil, $4
Creamy Pesto Pasta, Al dente pasta coated in a creamy pesto sauce made with fresh basil, pine nuts, garlic, Parmesan cheese, and olive oil, Pasta, Basil, Pine nuts, Garlic, Parmesan cheese, Olive oil, $2
Tiramisu Delight, A classic Italian dessert made with layers of coffee-soaked ladyfingers, mascarpone cheese, cocoa powder, and a hint of liqueur, Ladyfingers, Mascarpone cheese, Cocoa powder, Coffee, Liqueur, $2


In [21]:
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Shared Variables: ['agent']
Task: Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Output in Menu format.
Subtasks Completed:
Subtask: Creative Writer(instruction="Generate only 5 Italian dish names and descriptions")
{'Agent Output': '1. Rustic Tuscan Chicken: A hearty and flavorful dish inspired by the Tuscan countryside. Tender chicken cooked in a rich tomato and herb sauce, served with a side of creamy polenta and roasted vegetables. A comforting and satisfying meal that brings the taste of Italy to your table.\n2. Classic Margherita Pizza: A traditional Italian pizza featuring a thin crust topped with fresh tomatoes, mozzarella cheese, basil leaves, and a drizzle of olive oil. A timeless favorite that capt

In [22]:
# you can summarise the subtasks_history for the next task
my_agent.summarise_subtasks_completed('Generate the 3 Italian dishes for Gordon Ramsay in menu format')

1. Classic Margherita Pizza
Description: A traditional Italian pizza featuring a thin crust topped with fresh tomatoes, mozzarella cheese, basil leaves, and a drizzle of olive oil.
Ingredients: Tomatoes, Mozzarella cheese, Basil leaves, Olive oil
Price: $4

2. Creamy Pesto Pasta
Description: Al dente pasta coated in a creamy pesto sauce made with fresh basil, pine nuts, garlic, Parmesan cheese, and olive oil.
Ingredients: Pasta, Basil, Pine nuts, Garlic, Parmesan cheese, Olive oil
Price: $2

3. Tiramisu Delight
Description: A classic Italian dessert made with layers of coffee-soaked ladyfingers, mascarpone cheese, cocoa powder, and a hint of liqueur.
Ingredients: Ladyfingers, Mascarpone cheese, Cocoa powder, Coffee, Liqueur
Price: $2


In [23]:
# subtask has been summarised, helping to reduce the context significantly for later task generation
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Shared Variables: ['agent']
Task: Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Output in Menu format.
Subtasks Completed:
Subtask: Current Results for 'Generate the 3 Italian dishes for Gordon Ramsay in menu format'
1. Classic Margherita Pizza
Description: A traditional Italian pizza featuring a thin crust topped with fresh tomatoes, mozzarella cheese, basil leaves, and a drizzle of olive oil.
Ingredients: Tomatoes, Mozzarella cheese, Basil leaves, Olive oil
Price: $4

2. Creamy Pesto Pasta
Description: Al dente pasta coated in a creamy pesto sauce made with fresh basil, pine nuts, garlic, Parmesan cheese, and olive oil.
Ingredients: Pasta, Basil, Pine nuts, Garlic, Parmesan cheese, Olive oil
Price: $2

3. 

In [24]:
# we also allow you to manually remove the last item added to subtasks_completed, so you can revert your agent state if needed for backtracking
my_agent.remove_last_subtask()

Removed last subtask from subtasks_completed: ("Current Results for 'Generate the 3 Italian dishes for Gordon Ramsay in menu format'", '1. Classic Margherita Pizza\nDescription: A traditional Italian pizza featuring a thin crust topped with fresh tomatoes, mozzarella cheese, basil leaves, and a drizzle of olive oil.\nIngredients: Tomatoes, Mozzarella cheese, Basil leaves, Olive oil\nPrice: $4\n\n2. Creamy Pesto Pasta\nDescription: Al dente pasta coated in a creamy pesto sauce made with fresh basil, pine nuts, garlic, Parmesan cheese, and olive oil.\nIngredients: Pasta, Basil, Pine nuts, Garlic, Parmesan cheese, Olive oil\nPrice: $2\n\n3. Tiramisu Delight\nDescription: A classic Italian dessert made with layers of coffee-soaked ladyfingers, mascarpone cheese, cocoa powder, and a hint of liqueur.\nIngredients: Ladyfingers, Mascarpone cheese, Cocoa powder, Coffee, Liqueur\nPrice: $2')


In [25]:
# visualise your agent's status - should have empty subtasks_completed
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Shared Variables: ['agent']
Task: Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Output in Menu format.
Subtasks Completed: None
Is Task Completed: False
